In [1]:
from datasets import load_dataset, ClassLabel
from transformers import AutoTokenizer
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [2]:
pokemon_descriptions = load_dataset('../data/', delimiter=';')

Using custom data configuration data-c77318cc70bb336a


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 500.22it/s]


Dataset csv downloaded and prepared to C:/Users/CYBORGX/.cache/huggingface/datasets/csv/data-c77318cc70bb336a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 399.84it/s]


In [3]:
NUM_CLASSES = len(np.unique(pokemon_descriptions['train']['labels']))

In [4]:
# new_features = pokemon_descriptions['train'].features.copy()
# new_features["labels"] = ClassLabel(num_classes=len(np.unique(pokemon_descriptions['train']['labels'])))
# pokemon_descriptions = pokemon_descriptions.cast(new_features)

In [5]:
pokemon_descriptions

DatasetDict({
    train: Dataset({
        features: ['text', 'name', 'labels'],
        num_rows: 10789
    })
})

Train split stuff

In [6]:
split_pokemon_descriptions = pokemon_descriptions['train'].train_test_split(test_size=0.2, shuffle=True)

Parameter 'generator'=Generator(PCG64) of the transform datasets.arrow_dataset.Dataset.train_test_split couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [7]:
split_pokemon_descriptions

DatasetDict({
    train: Dataset({
        features: ['text', 'name', 'labels'],
        num_rows: 8631
    })
    test: Dataset({
        features: ['text', 'name', 'labels'],
        num_rows: 2158
    })
})

Tokenization

In [8]:
tokenizer_bert = AutoTokenizer.from_pretrained("bert-base-uncased")

In [9]:
tokenized_pokemon_descriptions = split_pokemon_descriptions.map(lambda example: tokenizer_bert(example['text'], truncation=True, padding=True), batched=True)

tokenized_pokemon_descriptions['train'] = tokenized_pokemon_descriptions['train'].remove_columns(['text', 'name'])
tokenized_pokemon_descriptions['test'] = tokenized_pokemon_descriptions['test'].remove_columns(['text', 'name'])

 67%|██████▋   | 2/3 [00:00<00:00,  8.25ba/s]


Fine tuning

In [10]:
tokenized_pokemon_descriptions

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8631
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2158
    })
})

In [11]:

from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from datasets import load_metric

In [12]:
def compute_metrics(eval_pred):
    """ Computes the metrics given a tuple of (logits, labels) """

    load_accuracy = load_metric("accuracy")
    precision_metric = load_metric('precision')
    recall_metric = load_metric('recall')
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(
        predictions=predictions, references=labels)["accuracy"]
    precision = precision_metric.compute(
        predictions=predictions, references=labels, average="macro", zero_division=0)["precision"]
    recall = recall_metric.compute(
        predictions=predictions, references=labels, average="macro", zero_division=0)["recall"]
    f1 = load_f1.compute(predictions=predictions,
                         references=labels, average="macro")["f1"]

    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }


In [13]:
def print_confusion_matrix(labels, predictions):
    """ Prints a confusion matrix for given labels and predictions. """
    cm = confusion_matrix(labels, predictions)
    disp = ConfusionMatrixDisplay(
        confusion_matrix=cm, display_labels=list(id2label.values()))
    disp.plot()


In [14]:
from transformers import (
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    PreTrainedModel,
    Trainer,
    TrainingArguments,
    PreTrainedTokenizerFast
)


In [15]:
def setup_trainer_for_finetuning(
        model: PreTrainedModel,
        tokenizer: PreTrainedTokenizerFast) -> Trainer:

    finetuning_args = TrainingArguments(
        output_dir="./results",
        learning_rate=1e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=10,
        weight_decay=0.01,
        warmup_ratio=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        seed=42,
        dataloader_num_workers=6,
        load_best_model_at_end=True
    )

    return Trainer(
        model=model,
        args=finetuning_args,
        train_dataset=tokenized_pokemon_descriptions['train'],
        eval_dataset=tokenized_pokemon_descriptions['test'],
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metrics
    )


Model

In [16]:
bert_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=NUM_CLASSES)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [17]:
trainer = setup_trainer_for_finetuning(model=bert_model, tokenizer=tokenizer_bert)

In [18]:
trainer.train()

c:\Users\CYBORGX\anaconda3\envs\birdclef\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8631
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5400
  Number of trainable parameters = 109598359
  9%|▉         | 500/5400 [02:20<19:31,  4.18it/s] 

{'loss': 4.9967, 'learning_rate': 9.165731387953612e-06, 'epoch': 0.93}


 10%|█         | 540/5400 [02:31<19:21,  4.19it/s]***** Running Evaluation *****
  Num examples = 2158
  Batch size = 16
C:\Users\CYBORGX\AppData\Local\Temp\ipykernel_12264\3270434908.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")
                                                  
 10%|█         | 540/5400 [02:58<19:21,  4.19it/s]Saving model checkpoint to ./results\checkpoint-540
Configuration saved in ./results\checkpoint-540\config.json


{'eval_loss': 4.8011860847473145, 'eval_accuracy': 0.05792400370713624, 'eval_f1': 0.027275958091292383, 'eval_precision': 0.03710849614775308, 'eval_recall': 0.04716188812677515, 'eval_runtime': 26.9047, 'eval_samples_per_second': 80.209, 'eval_steps_per_second': 5.018, 'epoch': 1.0}


Model weights saved in ./results\checkpoint-540\pytorch_model.bin
tokenizer config file saved in ./results\checkpoint-540\tokenizer_config.json
Special tokens file saved in ./results\checkpoint-540\special_tokens_map.json
 19%|█▊        | 1000/5400 [05:31<20:55,  3.50it/s]  

...

{'eval_loss': 2.8929872512817383, 'eval_accuracy': 0.5347544022242817, 'eval_f1': 0.5207951767211005, 'eval_precision': 0.6321200985362881, 'eval_recall': 0.5220100753233059, 'eval_runtime': 26.6159, 'eval_samples_per_second': 81.079, 'eval_steps_per_second': 5.072, 'epoch': 10.0}


Model weights saved in ./results\checkpoint-5400\pytorch_model.bin
tokenizer config file saved in ./results\checkpoint-5400\tokenizer_config.json
Special tokens file saved in ./results\checkpoint-5400\special_tokens_map.json
Deleting older checkpoint [results\checkpoint-4320] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results\checkpoint-5400 (score: 2.8929872512817383).
100%|██████████| 5400/5400 [32:43<00:00,  4.28it/s]Deleting older checkpoint [results\checkpoint-4860] due to args.save_total_limit
Deleting older checkpoint [results\checkpoint-5400] due to args.save_total_limit


{'train_runtime': 1963.7595, 'train_samples_per_second': 43.951, 'train_steps_per_second': 2.75, 'train_loss': 3.5502230043764467, 'epoch': 10.0}


100%|██████████| 5400/5400 [32:44<00:00,  2.75it/s]


TrainOutput(global_step=5400, training_loss=3.5502230043764467, metrics={'train_runtime': 1963.7595, 'train_samples_per_second': 43.951, 'train_steps_per_second': 2.75, 'train_loss': 3.5502230043764467, 'epoch': 10.0})

In [20]:
trainer.save_model('./results/saved-models/')

Saving model checkpoint to ./results/saved-models/
Configuration saved in ./results/saved-models/config.json
Model weights saved in ./results/saved-models/pytorch_model.bin
tokenizer config file saved in ./results/saved-models/tokenizer_config.json
Special tokens file saved in ./results/saved-models/special_tokens_map.json
